In [ ]:
from map import Map, Marker
import db

In [ ]:
boundaries = db.query(
    """
select st_asgeojson(geom) from boundaries
where id = 'chicago' 
"""
)

In [ ]:
wards = db.query(
    """
select id, st_asgeojson(geom)
from stg_chicago__wards
"""
)

In [ ]:
from ipyleaflet import GeoJSON, WidgetControl
from ipywidgets import Label, Layout

import json

m = Map((41.83, -87.6))

for row in boundaries.fetchall():
    geo_json = GeoJSON(data=json.loads(row[0]), style={"color": "blue", "weight": 3})
    m.add(geo_json)

label = Label(layout=Layout(width="100%"))
m.add(WidgetControl(widget=label, position="topright"))


def hover_handler(event=None, feature=None, id=None, properties=None):
    label.value = f'Ward: {feature["properties"]["id"]}'


for row in wards.fetchall():
    data = json.loads(row[1])
    data["properties"] = {}
    data["properties"]["id"] = str(row[0])
    geo_json = GeoJSON(
        data=data,
        style={"weight": 1},
        hover_style={"fillOpacity": 0.5},
    )
    m.add(geo_json)

    geo_json.on_hover(hover_handler)

m